<a href="https://colab.research.google.com/github/hyunaeee/KU-SW-Academy/blob/main/0907.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ORM
ORM Core
Scheme(Table, Column, Foriegn...), Types(Integer, Text, ...), ...
Engine - Connection Pool, Dialect - DB API - (R)DBMS - Database
   ^

Only SQLite
ORM               ORM Core             RDBMS
ORM Object      Table Object           TABLE
                                     ---------
in-memory       ------------
독립                  ?                   독립
             Table.Foreign_key      TABLE--PK/FK--TABLE
Object--relationship--Object
Object1.childnodes[] ==> Object2


In [ ]:
from sqlalchemy.orm import declarative_base

In [ ]:
base = declarative_base()

In [ ]:
base.metadata.tables # <- Core Table 객체-RDMBS

FacadeDict({})

In [ ]:
base.registry # <- Object 등록

In [ ]:
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [ ]:
base.metadata.tables

FacadeDict({})

In [ ]:
base.metadata.remove(base.metadata.tables['USER'])
base.registry.dispose()

In [ ]:
class User(base):
    __tablename__ = 'USER'
    # obj->테이블이름   => DBMS 사용될 Table이름
    pk = Column('PK', Integer, primary_key=True)
    # obj->pk        => DBMS 해당 Table 사용되는 Column
    name = Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [ ]:
user1 = User(name='아무거나')

TypeError: ignored

In [ ]:
type(user1), type(base.metadata.tables['USER'])

In [ ]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
base.metadata.create_all(engine)

In [ ]:
# session => engine.connect().execute() => DBMS

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(engine)
sess = Session()
# in-memory objects -> RDBMS

In [ ]:
sess.add(user1)

In [ ]:
user1

In [ ]:
sess.commit()
# session 관찰하고 있는 objects을 반영 => DBMS Commit()

In [ ]:
user1

In [ ]:
sess.is_modified(user1)

In [ ]:
sess.dirty

In [ ]:
user1.name = '다른값'

In [ ]:
user1

In [ ]:
sess.dirty

In [ ]:
sess.is_modified(user1)

In [ ]:
sess.commit()

In [ ]:
user1

In [ ]:
sess.add_all([User(name='2'), User(name='3')])

In [ ]:
sess.commit()

In [ ]:
userList = sess.query(User).all()

In [ ]:
len(userList)

In [ ]:
user1 is userList[0]

In [ ]:
userList[1].name = '다른값2'

In [ ]:
sess.dirty

In [ ]:
sess.commit()

In [ ]:
userList[1]

In [ ]:
[user for user in userList if user.name == '3'][0] is userList[2]

In [ ]:
sess.query(User).where(User.name == '3').one() is userList[2]

In [ ]:
class Address(base):
    __tablename__ = 'ADDRESS'

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey('USER.PK'))

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, FK:{self.fk}'

In [ ]:
base.metadata.create_all(engine)

In [ ]:
sess.add(Address(name='주소1', fk=user1.pk))

In [ ]:
sess.commit()

In [ ]:
address1 = sess.query(Address).one()

In [ ]:
for user in userList:
    if user.pk == address1.fk:
        print(user, address1)

In [ ]:
engine.dispose()
base.metadata.clear()
base.registry.dispose()

In [ ]:
CITY, SUPPLIER, PART, SELLS => ORM 만들어보세요(10시10분까지)

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)
Session = sessionmaker(engine)
sess = Session()

In [ ]:
class City(base):
    __tablename__ = 'CITY'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Supplier(base):
    __tablename__ = 'SUPPLIER'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))

class Part(base):
    __tablename__ = 'PART'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Sells(base):
    __tablename__ = 'SELLS'
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))

In [ ]:
base.metadata.create_all(engine)

In [ ]:
sess.add_all([City(name='도시1'), City(name='도시2'),
              Part(name='부품1'), Part(name='부품2'),
              Supplier(name='지점1'), Supplier(name='지점2'),])

In [ ]:
sess.commit()

In [ ]:
pk = sess.query(City).where(City.pk == 1).one().pk
sess.query(Supplier).filter(Supplier.pk == 1).one().fk = pk

In [ ]:
sess.query(Supplier).all()[0].fk

In [ ]:
sess.query(City.name, Supplier.name).join(Supplier).all()

In [ ]:
from sqlalchemy.orm import relationship

In [ ]:
base.registry.dispose()

In [ ]:
class City(base):
    __tablename__ = 'CITY'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    nodes = relationship('Supplier', back_populates='city',
                         uselist=True)

class Supplier(base):
    __tablename__ = 'SUPPLIER'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))
    city = relationship('City', back_populates='nodes')

class Part(base):
    __tablename__ = 'PART'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Sells(base):
    __tablename__ = 'SELLS'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))

In [ ]:
city = sess.query(City).where(City.pk==1).one()

In [ ]:
city.nodes[0] is sess.query(Supplier).where(Supplier.fk==1).one()

In [ ]:
len(city.nodes)

In [ ]:
city.nodes.append(Supplier(name='지점2'))

In [ ]:
sess.is_modified(city)

In [ ]:
sess.commit()

In [ ]:
len(city.nodes)

In [ ]:
city.nodes[-1].name

In [ ]:
suppliers = sess.query(Supplier).all()

In [ ]:
sess.close()

In [ ]:
Session = sessionmaker(engine)
sess = Session()

In [ ]:
suppliers[0].city.name

In [ ]:
sell.suppliers.city.nodes.sells

In [ ]:
sess.rollback()

In [ ]:
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

In [ ]:
engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [ ]:
base.registry.dispose()
base.metadata.clear()

In [ ]:
class Artist(base):
    __tablename__ = 'ARTIST'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    albums = relationship('Album', back_populates='artist',
                          uselist=True)
    def addAlbum(self, name):
        album = Album(name=name)
        sess.add(album)
        sess.commit()
        self.albums.append(album)
    def delAlbum(self, name):
        album = sess.query(Album).filter(Album.name=name).one()
        self.albums.index(album)

    def __repr__(self):
        return f'{self.name}, {len(self.albums)}'

class Album(base):
    __tablename__ = 'ALBUM'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('ARTIST.PK'))
    artist = relationship('Artist', back_populates='albums')

    def __repr__(self):
        return f'{self.name}, {self.artist.name}'

In [ ]:
rst = sess.query(Album).filter(Album.name=='앨범1')

In [ ]:
album1 = rst.one()

In [ ]:
artist1.albums.index(album1)

In [ ]:
base.metadata.create_all(engine)

In [ ]:
artist1 = Artist(name='가수1')
sess.add(artist1)
sess.commit()

In [ ]:
artist1.addAlbum('앨범1')

In [ ]:
artist1.albums[0]

In [ ]:
앨범을 삭제하는 코드를 만들기(11시20분)

In [ ]:
sess.rollback()

engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [ ]:
# 기존에서 가져오기
base.metadata.reflect(engine)
base.metadata.tables['테이블이름']
class 어쩌고(base):
    __table__ = base.metadata.tables['테이블이름']
    pk =
    relationship
DB, PK/FK 설정
relationship <= PK/FK, 객체, DB X

In [ ]:
base.registry.dispose()

In [ ]:
# Artist 1:N Ablums
# Post N:M Hashtag => 별도의 Table
class Post(base):
    __tablename__ = 'POST'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text)
    tags = relationship('Tags', back_populates='post', uselist=True)

    def addTags(self, tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = Hashtag(name=tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1=self.pk, fk2=tag.pk))
            sess.commit()
            tag.cnt += 1
    # tag 삭제
    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'

class Hashtag(base):
    __tablename__ = 'HASHTAG'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag',
                         uselist=True)
    # +1, -1 함수
    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'

class Tags(base):
    __tablename__ = 'TAGS'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', None, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

# Object/Class
# Post           Tags           Hashtag
# tags----      post/hashtag    ----posts
# Posts.tags[0/1/2/3/..].hashtag -----------> Hashtag

In [ ]:
base.metadata.create_all(engine)

In [ ]:
post1 = Post(content='내용1')
sess.add(post1)
sess.commit()

In [ ]:
tag1 = Hashtag(name='해시태그1')
sess.add(tag1)
sess.commit()

In [ ]:
sess.add(Tags(fk1=post1.pk, fk2=tag1.pk))
sess.commit()

NameError: ignored

In [ ]:
len(post1.tags), post1.tags[0].hashtag.name

In [ ]:
len(tag1.posts), tag1.posts[0].post.content

In [ ]:
post2 = Post(content='내용2')
sess.add(post2)
sess.commit()

In [ ]:
post2.addTags(['해시태그2', '해시태그3'])

In [ ]:
for tag in post2.tags:
    print(tag.hashtag)

In [ ]:
post3 = Post(content='내용3')
sess.add(post3)
sess.commit()

In [ ]:
post3.addTags(['해시태그1', '해시태그2'])

In [ ]:
tag1 = sess.query(Hashtag).where(Hashtag.name == '해시태그2').one()

In [ ]:
for post in tag1.posts:
    print(post.post.content)

In [ ]:
tag1

In [ ]:
ORM
ORM Core
Scheme(Table, Column, Foriegn...), Types(Integer, Text, ...), ...
Engine - Connection Pool, Dialect - DB API - (R)DBMS - Database
   ^

Only SQLite
ORM               ORM Core             RDBMS
ORM Object      Table Object           TABLE
                                     ---------
in-memory       ------------
독립                  ?                   독립
             Table.Foreign_key      TABLE--PK/FK--TABLE
Object--relationship--Object
Object1.childnodes[] ==> Object2
from sqlalchemy.orm import declarative_base
base = declarative_base()
base.metadata.tables # <- Core Table 객체-RDMBS
FacadeDict({})
base.registry # <- Object 등록
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
base.metadata.tables
FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>), schema=None)})
base.metadata.remove(base.metadata.tables['USER'])
base.registry.dispose()
class User(base):
    __tablename__ = 'USER'
    # obj->테이블이름   => DBMS 사용될 Table이름
    pk = Column('PK', Integer, primary_key=True)
    # obj->pk        => DBMS 해당 Table 사용되는 Column
    name = Column('NAME', Text)

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'
user1 = User(name='아무거나')
type(user1), type(base.metadata.tables['USER'])
(__main__.User, sqlalchemy.sql.schema.Table)
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
base.metadata.create_all(engine)
2023-09-07 09:24:28,808 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:24:28,810 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:24:28,814 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:28,815 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-07 09:24:28,816 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:24:28,817 INFO sqlalchemy.engine.Engine
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	PRIMARY KEY ("PK")
)


2023-09-07 09:24:28,818 INFO sqlalchemy.engine.Engine [no key 0.00108s] ()
2023-09-07 09:24:28,819 INFO sqlalchemy.engine.Engine COMMIT
# session => engine.connect().execute() => DBMS
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(engine)
sess = Session()
# in-memory objects -> RDBMS
sess.add(user1)
user1
PK:None, NAME:아무거나
sess.commit()
# session 관찰하고 있는 objects을 반영 => DBMS Commit()
2023-09-07 09:29:23,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:29:23,514 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?)
2023-09-07 09:29:23,515 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('아무거나',)
2023-09-07 09:29:23,517 INFO sqlalchemy.engine.Engine COMMIT
user1
2023-09-07 09:29:27,883 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:29:27,890 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME"
FROM "USER"
WHERE "USER"."PK" = ?
2023-09-07 09:29:27,891 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (1,)
PK:1, NAME:아무거나
sess.is_modified(user1)
False
sess.dirty
IdentitySet([])
user1.name = '다른값'
user1
PK:1, NAME:다른값
sess.dirty
IdentitySet([PK:1, NAME:다른값])
sess.is_modified(user1)
True
sess.commit()
2023-09-07 09:32:44,953 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:32:44,957 INFO sqlalchemy.engine.Engine [generated in 0.00438s] ('다른값', 1)
2023-09-07 09:32:44,959 INFO sqlalchemy.engine.Engine COMMIT
user1
2023-09-07 09:32:54,246 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:32:54,250 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME"
FROM "USER"
WHERE "USER"."PK" = ?
2023-09-07 09:32:54,251 INFO sqlalchemy.engine.Engine [cached since 206.4s ago] (1,)
PK:1, NAME:다른값
sess.add_all([User(name='2'), User(name='3')])
sess.commit()
2023-09-07 09:34:16,792 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:16,795 INFO sqlalchemy.engine.Engine [generated in 0.00036s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('2',)
2023-09-07 09:34:16,797 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 09:34:16,799 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('3',)
2023-09-07 09:34:16,801 INFO sqlalchemy.engine.Engine COMMIT
userList = sess.query(User).all()
2023-09-07 09:34:36,031 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:34:36,036 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME"
FROM "USER"
2023-09-07 09:34:36,039 INFO sqlalchemy.engine.Engine [generated in 0.00278s] ()
len(userList)
3
user1 is userList[0]
True
userList[1].name = '다른값2'
sess.dirty
IdentitySet([PK:2, NAME:다른값2])
sess.commit()
2023-09-07 09:35:34,079 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-07 09:35:34,085 INFO sqlalchemy.engine.Engine [cached since 169.1s ago] ('다른값2', 2)
2023-09-07 09:35:34,088 INFO sqlalchemy.engine.Engine COMMIT
userList[1]
2023-09-07 09:35:40,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:35:40,021 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME"
FROM "USER"
WHERE "USER"."PK" = ?
2023-09-07 09:35:40,023 INFO sqlalchemy.engine.Engine [cached since 372.1s ago] (2,)
PK:2, NAME:다른값2
[user for user in userList if user.name == '3'][0] is userList[2]
True
sess.query(User).where(User.name == '3').one() is userList[2]
2023-09-07 09:39:00,275 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME"
FROM "USER"
WHERE "USER"."NAME" = ?
2023-09-07 09:39:00,279 INFO sqlalchemy.engine.Engine [cached since 4.094s ago] ('3',)
True
class Address(base):
    __tablename__ = 'ADDRESS'

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey('USER.PK'))

    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, FK:{self.fk}'
base.metadata.create_all(engine)
2023-09-07 09:44:59,306 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 09:44:59,312 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-07 09:44:59,315 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:44:59,316 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ADDRESS")
2023-09-07 09:44:59,317 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:44:59,318 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ADDRESS")
2023-09-07 09:44:59,318 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 09:44:59,320 INFO sqlalchemy.engine.Engine
CREATE TABLE "ADDRESS" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	"FK" INTEGER,
	PRIMARY KEY ("PK"),
	FOREIGN KEY("FK") REFERENCES "USER" ("PK")
)


2023-09-07 09:44:59,321 INFO sqlalchemy.engine.Engine [no key 0.00105s] ()
2023-09-07 09:44:59,322 INFO sqlalchemy.engine.Engine COMMIT
sess.add(Address(name='주소1', fk=user1.pk))
sess.commit()
2023-09-07 09:45:46,538 INFO sqlalchemy.engine.Engine INSERT INTO "ADDRESS" ("NAME", "FK") VALUES (?, ?)
2023-09-07 09:45:46,540 INFO sqlalchemy.engine.Engine [generated in 0.00290s] ('주소1', 1)
2023-09-07 09:45:46,543 INFO sqlalchemy.engine.Engine COMMIT
address1 = sess.query(Address).one()
2023-09-07 09:46:12,918 INFO sqlalchemy.engine.Engine SELECT "ADDRESS"."PK" AS "ADDRESS_PK", "ADDRESS"."NAME" AS "ADDRESS_NAME", "ADDRESS"."FK" AS "ADDRESS_FK"
FROM "ADDRESS"
2023-09-07 09:46:12,922 INFO sqlalchemy.engine.Engine [cached since 14.02s ago] ()
for user in userList:
    if user.pk == address1.fk:
        print(user, address1)
PK:1, NAME:다른값 PK:1, NAME:주소1, FK:1
engine.dispose()
base.metadata.clear()
base.registry.dispose()
CITY, SUPPLIER, PART, SELLS => ORM 만들어보세요(10시10분까지)
engine = create_engine('sqlite:///:memory:', echo=True)
Session = sessionmaker(engine)
sess = Session()
class City(base):
    __tablename__ = 'CITY'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Supplier(base):
    __tablename__ = 'SUPPLIER'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))

class Part(base):
    __tablename__ = 'PART'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Sells(base):
    __tablename__ = 'SELLS'
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))
base.metadata.create_all(engine)
2023-09-07 10:23:04,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:23:04,935 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CITY")
2023-09-07 10:23:04,936 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,937 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CITY")
2023-09-07 10:23:04,938 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,940 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SUPPLIER")
2023-09-07 10:23:04,940 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,941 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SUPPLIER")
2023-09-07 10:23:04,942 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,945 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PART")
2023-09-07 10:23:04,946 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,947 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PART")
2023-09-07 10:23:04,948 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,948 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELLS")
2023-09-07 10:23:04,949 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,949 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELLS")
2023-09-07 10:23:04,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:23:04,951 INFO sqlalchemy.engine.Engine
CREATE TABLE "CITY" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	PRIMARY KEY ("PK")
)


2023-09-07 10:23:04,951 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2023-09-07 10:23:04,952 INFO sqlalchemy.engine.Engine
CREATE TABLE "PART" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	PRIMARY KEY ("PK")
)


2023-09-07 10:23:04,952 INFO sqlalchemy.engine.Engine [no key 0.00036s] ()
2023-09-07 10:23:04,953 INFO sqlalchemy.engine.Engine
CREATE TABLE "SUPPLIER" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	"FK" INTEGER,
	PRIMARY KEY ("PK"),
	FOREIGN KEY("FK") REFERENCES "CITY" ("PK")
)


2023-09-07 10:23:04,953 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2023-09-07 10:23:04,954 INFO sqlalchemy.engine.Engine
CREATE TABLE "SELLS" (
	"PK" INTEGER NOT NULL,
	"FK1" INTEGER,
	"FK2" INTEGER,
	PRIMARY KEY ("PK"),
	FOREIGN KEY("FK1") REFERENCES "SUPPLIER" ("PK"),
	FOREIGN KEY("FK2") REFERENCES "PART" ("PK")
)


2023-09-07 10:23:04,955 INFO sqlalchemy.engine.Engine [no key 0.00041s] ()
2023-09-07 10:23:04,956 INFO sqlalchemy.engine.Engine COMMIT
sess.add_all([City(name='도시1'), City(name='도시2'),
              Part(name='부품1'), Part(name='부품2'),
              Supplier(name='지점1'), Supplier(name='지점2'),])
sess.commit()
2023-09-07 10:23:14,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:23:14,067 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:23:14,072 INFO sqlalchemy.engine.Engine [generated in 0.00055s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('도시1',)
2023-09-07 10:23:14,078 INFO sqlalchemy.engine.Engine INSERT INTO "CITY" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:23:14,078 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('도시2',)
2023-09-07 10:23:14,080 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:23:14,080 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('부품1',)
2023-09-07 10:23:14,080 INFO sqlalchemy.engine.Engine INSERT INTO "PART" ("NAME") VALUES (?) RETURNING "PK"
2023-09-07 10:23:14,081 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('부품2',)
2023-09-07 10:23:14,082 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("NAME", "FK") VALUES (?, ?) RETURNING "PK"
2023-09-07 10:23:14,082 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('지점1', None)
2023-09-07 10:23:14,083 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("NAME", "FK") VALUES (?, ?) RETURNING "PK"
2023-09-07 10:23:14,083 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('지점2', None)
2023-09-07 10:23:14,084 INFO sqlalchemy.engine.Engine COMMIT
pk = sess.query(City).where(City.pk == 1).one().pk
sess.query(Supplier).filter(Supplier.pk == 1).one().fk = pk
2023-09-07 10:24:29,873 INFO sqlalchemy.engine.Engine SELECT "CITY"."PK" AS "CITY_PK", "CITY"."NAME" AS "CITY_NAME"
FROM "CITY"
WHERE "CITY"."PK" = ?
2023-09-07 10:24:29,874 INFO sqlalchemy.engine.Engine [generated in 0.00198s] (1,)
2023-09-07 10:24:29,878 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."PK" AS "SUPPLIER_PK", "SUPPLIER"."NAME" AS "SUPPLIER_NAME", "SUPPLIER"."FK" AS "SUPPLIER_FK"
FROM "SUPPLIER"
WHERE "SUPPLIER"."PK" = ?
2023-09-07 10:24:29,879 INFO sqlalchemy.engine.Engine [cached since 69.02s ago] (1,)
sess.query(Supplier).all()[0].fk
2023-09-07 10:24:47,880 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."PK" AS "SUPPLIER_PK", "SUPPLIER"."NAME" AS "SUPPLIER_NAME", "SUPPLIER"."FK" AS "SUPPLIER_FK"
FROM "SUPPLIER"
2023-09-07 10:24:47,886 INFO sqlalchemy.engine.Engine [cached since 4.519s ago] ()
1
sess.query(City.name, Supplier.name).join(Supplier).all()
2023-09-07 10:25:33,095 INFO sqlalchemy.engine.Engine SELECT "CITY"."NAME" AS "CITY_NAME", "SUPPLIER"."NAME" AS "SUPPLIER_NAME"
FROM "CITY" JOIN "SUPPLIER" ON "CITY"."PK" = "SUPPLIER"."FK"
2023-09-07 10:25:33,098 INFO sqlalchemy.engine.Engine [generated in 0.00339s] ()
[('도시1', '지점1')]
from sqlalchemy.orm import relationship
base.registry.dispose()
class City(base):
    __tablename__ = 'CITY'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    nodes = relationship('Supplier', back_populates='city',
                         uselist=True)

class Supplier(base):
    __tablename__ = 'SUPPLIER'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('CITY.PK'))
    city = relationship('City', back_populates='nodes')

class Part(base):
    __tablename__ = 'PART'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)

class Sells(base):
    __tablename__ = 'SELLS'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('SUPPLIER.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('PART.PK'))
city = sess.query(City).where(City.pk==1).one()
2023-09-07 10:37:01,845 INFO sqlalchemy.engine.Engine SELECT "CITY"."PK" AS "CITY_PK", "CITY"."NAME" AS "CITY_NAME"
FROM "CITY"
WHERE "CITY"."PK" = ?
2023-09-07 10:37:01,850 INFO sqlalchemy.engine.Engine [generated in 0.00482s] (1,)
city.nodes[0] is sess.query(Supplier).where(Supplier.fk==1).one()
2023-09-07 10:37:28,848 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."PK" AS "SUPPLIER_PK", "SUPPLIER"."NAME" AS "SUPPLIER_NAME", "SUPPLIER"."FK" AS "SUPPLIER_FK"
FROM "SUPPLIER"
WHERE "SUPPLIER"."FK" = ?
2023-09-07 10:37:28,853 INFO sqlalchemy.engine.Engine [generated in 0.00568s] (1,)
True
len(city.nodes)
1
city.nodes.append(Supplier(name='지점2'))
sess.is_modified(city)
True
sess.commit()
2023-09-07 10:39:30,954 INFO sqlalchemy.engine.Engine INSERT INTO "SUPPLIER" ("NAME", "FK") VALUES (?, ?)
2023-09-07 10:39:30,955 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('지점2', 1)
2023-09-07 10:39:30,958 INFO sqlalchemy.engine.Engine COMMIT
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In [120], line 1
----> 1 sess.commit()

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sqlalchemy/orm/session.py:1923, in Session.commit(self)
   1920 if trans is None:
   1921     trans = self._autobegin_t()
-> 1923 trans.commit(_to_root=True)

File <string>:2, in commit(self, _to_root)

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sqlalchemy/orm/state_changes.py:139, in _StateChange.declare_states.<locals>._go(fn, self, *arg, **kw)
    137 self._next_state = _StateChangeStates.CHANGE_IN_PROGRESS
    138 try:
--> 139     ret_value = fn(self, *arg, **kw)
    140 except:
    141     raise

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sqlalchemy/orm/session.py:1251, in SessionTransaction.commit(self, _to_root)
   1248     self._state = SessionTransactionState.COMMITTED
   1249     self.session.dispatch.after_commit(self.session)
-> 1251     self._remove_snapshot()
   1253 with self._expect_state(SessionTransactionState.CLOSED):
   1254     self.close()

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sqlalchemy/orm/session.py:1075, in SessionTransaction._remove_snapshot(self)
   1073 if not self.nested and self.session.expire_on_commit:
   1074     for s in self.session.identity_map.all_states():
-> 1075         s._expire(s.dict, self.session.identity_map._modified)
   1077     statelib.InstanceState._detach_states(
   1078         list(self._deleted), self.session
   1079     )
   1080     self._deleted.clear()

File /opt/homebrew/anaconda3/lib/python3.9/site-packages/sqlalchemy/orm/state.py:744, in InstanceState._expire(self, dict_, modified_set)
    741 for key in self.manager._all_key_set.intersection(dict_):
    742     del dict_[key]
--> 744 self.manager.dispatch.expire(self, None)

AttributeError: 'NoneType' object has no attribute 'expire'
len(city.nodes)
2
city.nodes[-1].name
'지점2'
suppliers = sess.query(Supplier).all()
2023-09-07 10:42:32,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:42:32,585 INFO sqlalchemy.engine.Engine SELECT "SUPPLIER"."PK" AS "SUPPLIER_PK", "SUPPLIER"."NAME" AS "SUPPLIER_NAME", "SUPPLIER"."FK" AS "SUPPLIER_FK"
FROM "SUPPLIER"
2023-09-07 10:42:32,586 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ()
sess.close()
Session = sessionmaker(engine)
sess = Session()
suppliers[0].city.name
'도시1'
sell.suppliers.city.nodes.sells
'도시1'
sess.rollback()
engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()
engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()
base.registry.dispose()
base.metadata.clear()
class Artist(base):
    __tablename__ = 'ARTIST'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    albums = relationship('Album', back_populates='artist',
                          uselist=True)
    def addAlbum(self, name):
        album = Album(name=name)
        sess.add(album)
        sess.commit()
        self.albums.append(album)
    def delAlbum(self, name):
        album = sess.query(Album).filter(Album.name=name).one()
        self.albums.index(album)

    def __repr__(self):
        return f'{self.name}, {len(self.albums)}'

class Album(base):
    __tablename__ = 'ALBUM'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', Integer, ForeignKey('ARTIST.PK'))
    artist = relationship('Artist', back_populates='albums')

    def __repr__(self):
        return f'{self.name}, {self.artist.name}'
rst = sess.query(Album).filter(Album.name=='앨범1')
album1 = rst.one()
2023-09-07 11:23:57,065 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK"
FROM "ALBUM"
WHERE "ALBUM"."NAME" = ?
2023-09-07 11:23:57,068 INFO sqlalchemy.engine.Engine [generated in 0.00309s] ('앨범1',)
artist1.albums.index(album1)
0
base.metadata.create_all(engine)
2023-09-07 10:54:59,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:54:59,401 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ARTIST")
2023-09-07 10:54:59,403 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:54:59,405 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ARTIST")
2023-09-07 10:54:59,406 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:54:59,407 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ALBUM")
2023-09-07 10:54:59,408 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:54:59,409 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ALBUM")
2023-09-07 10:54:59,410 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 10:54:59,412 INFO sqlalchemy.engine.Engine
CREATE TABLE "ARTIST" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	PRIMARY KEY ("PK")
)


2023-09-07 10:54:59,412 INFO sqlalchemy.engine.Engine [no key 0.00080s] ()
2023-09-07 10:54:59,414 INFO sqlalchemy.engine.Engine
CREATE TABLE "ALBUM" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	"FK" INTEGER,
	PRIMARY KEY ("PK"),
	FOREIGN KEY("FK") REFERENCES "ARTIST" ("PK")
)


2023-09-07 10:54:59,414 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2023-09-07 10:54:59,415 INFO sqlalchemy.engine.Engine COMMIT
artist1 = Artist(name='가수1')
sess.add(artist1)
sess.commit()
2023-09-07 10:55:00,161 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:55:00,163 INFO sqlalchemy.engine.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2023-09-07 10:55:00,164 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ('가수1',)
2023-09-07 10:55:00,165 INFO sqlalchemy.engine.Engine COMMIT
artist1.addAlbum('앨범1')
2023-09-07 10:55:01,174 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:55:01,178 INFO sqlalchemy.engine.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2023-09-07 10:55:01,179 INFO sqlalchemy.engine.Engine [generated in 0.00161s] ('앨범1', None)
2023-09-07 10:55:01,181 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 10:55:01,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 10:55:01,184 INFO sqlalchemy.engine.Engine SELECT "ARTIST"."PK" AS "ARTIST_PK", "ARTIST"."NAME" AS "ARTIST_NAME"
FROM "ARTIST"
WHERE "ARTIST"."PK" = ?
2023-09-07 10:55:01,185 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (1,)
2023-09-07 10:55:01,187 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK"
FROM "ALBUM"
WHERE ? = "ALBUM"."FK"
2023-09-07 10:55:01,189 INFO sqlalchemy.engine.Engine [generated in 0.00142s] (1,)
artist1.albums[0]
2023-09-07 10:55:01,958 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME"
FROM "ALBUM"
WHERE "ALBUM"."PK" = ?
2023-09-07 10:55:01,960 INFO sqlalchemy.engine.Engine [generated in 0.00285s] (1,)
2023-09-07 10:55:01,963 INFO sqlalchemy.engine.Engine UPDATE "ALBUM" SET "FK"=? WHERE "ALBUM"."PK" = ?
2023-09-07 10:55:01,964 INFO sqlalchemy.engine.Engine [generated in 0.00090s] (1, 1)
2023-09-07 10:55:01,966 INFO sqlalchemy.engine.Engine SELECT "ALBUM"."PK" AS "ALBUM_PK", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK"
FROM "ALBUM"
WHERE "ALBUM"."PK" = ?
2023-09-07 10:55:01,966 INFO sqlalchemy.engine.Engine [generated in 0.00070s] (1,)
앨범1, 가수1
앨범을 삭제하는 코드를 만들기(11시20분)
sess.rollback()

engine.dispose()
sess.close()
base.registry.dispose()
base.metadata.clear()

engine = create_engine('sqlite:///:memory:', echo=True)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()
# 기존에서 가져오기
base.metadata.reflect(engine)
base.metadata.tables['테이블이름']
class 어쩌고(base):
    __table__ = base.metadata.tables['테이블이름']
    pk =
    relationship
DB, PK/FK 설정
relationship <= PK/FK, 객체, DB X
base.registry.dispose()
# Artist 1:N Ablums
# Post N:M Hashtag => 별도의 Table
class Post(base):
    __tablename__ = 'POST'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text)
    tags = relationship('Tags', back_populates='post', uselist=True)

    def addTags(self, tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = Hashtag(name=tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1=self.pk, fk2=tag.pk))
            sess.commit()
            tag.cnt += 1
    # tag 삭제
    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'

class Hashtag(base):
    __tablename__ = 'HASHTAG'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag',
                         uselist=True)
    # +1, -1 함수
    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'

class Tags(base):
    __tablename__ = 'TAGS'
    __table_args__ = {'extend_existing':True}
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', None, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

# Object/Class
# Post           Tags           Hashtag
# tags----      post/hashtag    ----posts
# Posts.tags[0/1/2/3/..].hashtag -----------> Hashtag
base.metadata.create_all(engine)
2023-09-07 11:53:59,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:53:59,398 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-07 11:53:59,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,402 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-07 11:53:59,403 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,403 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-07 11:53:59,404 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,405 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-07 11:53:59,405 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,406 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAGS")
2023-09-07 11:53:59,407 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,408 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TAGS")
2023-09-07 11:53:59,409 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-07 11:53:59,411 INFO sqlalchemy.engine.Engine
CREATE TABLE "POST" (
	"PK" INTEGER NOT NULL,
	"CONTENT" TEXT,
	PRIMARY KEY ("PK")
)


2023-09-07 11:53:59,411 INFO sqlalchemy.engine.Engine [no key 0.00036s] ()
2023-09-07 11:53:59,412 INFO sqlalchemy.engine.Engine
CREATE TABLE "HASHTAG" (
	"PK" INTEGER NOT NULL,
	"NAME" TEXT,
	"CNT" INTEGER DEFAULT '0',
	PRIMARY KEY ("PK")
)


2023-09-07 11:53:59,413 INFO sqlalchemy.engine.Engine [no key 0.00057s] ()
2023-09-07 11:53:59,414 INFO sqlalchemy.engine.Engine
CREATE TABLE "TAGS" (
	"PK" INTEGER NOT NULL,
	"FK1" INTEGER,
	"FK2" INTEGER,
	PRIMARY KEY ("PK"),
	FOREIGN KEY("FK1") REFERENCES "POST" ("PK"),
	FOREIGN KEY("FK2") REFERENCES "HASHTAG" ("PK")
)


2023-09-07 11:53:59,414 INFO sqlalchemy.engine.Engine [no key 0.00026s] ()
2023-09-07 11:53:59,414 INFO sqlalchemy.engine.Engine COMMIT
post1 = Post(content='내용1')
sess.add(post1)
sess.commit()
2023-09-07 11:39:17,871 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:39:17,872 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:39:17,873 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ('내용1',)
2023-09-07 11:39:17,873 INFO sqlalchemy.engine.Engine COMMIT
tag1 = Hashtag(name='해시태그1')
sess.add(tag1)
sess.commit()
2023-09-07 11:39:56,131 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:39:56,136 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?)
2023-09-07 11:39:56,138 INFO sqlalchemy.engine.Engine [generated in 0.00155s] ('해시태그1',)
2023-09-07 11:39:56,140 INFO sqlalchemy.engine.Engine COMMIT
sess.add(Tags(fk1=post1.pk, fk2=tag1.pk))
sess.commit()
2023-09-07 11:40:32,334 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:40:32,338 INFO sqlalchemy.engine.Engine [generated in 0.00398s] (1,)
2023-09-07 11:40:32,342 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:40:32,342 INFO sqlalchemy.engine.Engine [generated in 0.00090s] (1, 1)
2023-09-07 11:40:32,344 INFO sqlalchemy.engine.Engine COMMIT
len(post1.tags), post1.tags[0].hashtag.name
2023-09-07 11:41:01,148 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:41:01,153 INFO sqlalchemy.engine.Engine [cached since 28.82s ago] (1,)
(1, '해시태그1')
len(tag1.posts), tag1.posts[0].post.content
(1, '내용1')
post2 = Post(content='내용2')
sess.add(post2)
sess.commit()
2023-09-07 11:54:08,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:08,769 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:54:08,770 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ('내용2',)
2023-09-07 11:54:08,772 INFO sqlalchemy.engine.Engine COMMIT
post2.addTags(['해시태그2', '해시태그3'])
2023-09-07 11:54:11,987 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:11,993 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:54:11,994 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ('해시태그2',)
2023-09-07 11:54:11,997 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?) RETURNING "PK", "CNT"
2023-09-07 11:54:11,998 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ('해시태그2',)
2023-09-07 11:54:11,999 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:54:12,000 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:12,000 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:54:12,000 INFO sqlalchemy.engine.Engine [cached since 0.007117s ago] ('해시태그3',)
2023-09-07 11:54:12,001 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?) RETURNING "PK", "CNT"
2023-09-07 11:54:12,001 INFO sqlalchemy.engine.Engine [cached since 0.004593s ago] ('해시태그3',)
2023-09-07 11:54:12,002 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:54:12,003 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:12,003 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:54:12,004 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (1,)
2023-09-07 11:54:12,006 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:12,006 INFO sqlalchemy.engine.Engine [generated in 0.00041s] (1,)
2023-09-07 11:54:12,007 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:54:12,008 INFO sqlalchemy.engine.Engine [generated in 0.00047s] (1, 1)
2023-09-07 11:54:12,008 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:54:12,009 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:12,009 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:12,010 INFO sqlalchemy.engine.Engine [cached since 0.004054s ago] (1,)
2023-09-07 11:54:12,011 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "CNT"=? WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:12,011 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (1, 1)
2023-09-07 11:54:12,012 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:54:12,012 INFO sqlalchemy.engine.Engine [cached since 0.008648s ago] (1,)
2023-09-07 11:54:12,013 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:12,013 INFO sqlalchemy.engine.Engine [cached since 0.007482s ago] (2,)
2023-09-07 11:54:12,014 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:54:12,015 INFO sqlalchemy.engine.Engine [cached since 0.007397s ago] (1, 2)
2023-09-07 11:54:12,015 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:54:12,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:54:12,016 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:12,017 INFO sqlalchemy.engine.Engine [cached since 0.01097s ago] (2,)
for tag in post2.tags:
    print(tag.hashtag)
2023-09-07 11:54:46,933 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:46,937 INFO sqlalchemy.engine.Engine [generated in 0.00452s] (1,)
2023-09-07 11:54:46,943 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2"
FROM "TAGS"
WHERE ? = "TAGS"."FK2"
2023-09-07 11:54:46,944 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (1,)
1, 해시태그2, 1
2023-09-07 11:54:46,946 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:54:46,947 INFO sqlalchemy.engine.Engine [cached since 0.01388s ago] (2,)
2023-09-07 11:54:46,948 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2"
FROM "TAGS"
WHERE ? = "TAGS"."FK2"
2023-09-07 11:54:46,948 INFO sqlalchemy.engine.Engine [cached since 0.005303s ago] (2,)
2, 해시태그3, 1
post3 = Post(content='내용3')
sess.add(post3)
sess.commit()
2023-09-07 11:55:31,581 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?)
2023-09-07 11:55:31,584 INFO sqlalchemy.engine.Engine [cached since 82.81s ago] ('내용3',)
2023-09-07 11:55:31,585 INFO sqlalchemy.engine.Engine COMMIT
post3.addTags(['해시태그1', '해시태그2'])
2023-09-07 11:55:51,634 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:51,639 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:55:51,641 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] ('해시태그1',)
2023-09-07 11:55:51,643 INFO sqlalchemy.engine.Engine INSERT INTO "HASHTAG" ("NAME") VALUES (?) RETURNING "PK", "CNT"
2023-09-07 11:55:51,644 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] ('해시태그1',)
2023-09-07 11:55:51,644 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:55:51,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:51,646 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-07 11:55:51,646 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] ('해시태그2',)
2023-09-07 11:55:51,647 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?
2023-09-07 11:55:51,647 INFO sqlalchemy.engine.Engine [generated in 0.00035s] ('해시태그2',)
2023-09-07 11:55:51,648 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:55:51,649 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (2,)
2023-09-07 11:55:51,650 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:55:51,650 INFO sqlalchemy.engine.Engine [cached since 99.64s ago] (3,)
2023-09-07 11:55:51,651 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:55:51,651 INFO sqlalchemy.engine.Engine [cached since 99.64s ago] (2, 3)
2023-09-07 11:55:51,652 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:55:51,652 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:51,653 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:55:51,653 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (3,)
2023-09-07 11:55:51,654 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "CNT"=? WHERE "HASHTAG"."PK" = ?
2023-09-07 11:55:51,654 INFO sqlalchemy.engine.Engine [cached since 99.64s ago] (1, 3)
2023-09-07 11:55:51,655 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:55:51,655 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (2,)
2023-09-07 11:55:51,656 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:55:51,656 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (1,)
2023-09-07 11:55:51,657 INFO sqlalchemy.engine.Engine INSERT INTO "TAGS" ("FK1", "FK2") VALUES (?, ?)
2023-09-07 11:55:51,657 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (2, 1)
2023-09-07 11:55:51,658 INFO sqlalchemy.engine.Engine COMMIT
2023-09-07 11:55:51,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-07 11:55:51,659 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."PK" = ?
2023-09-07 11:55:51,659 INFO sqlalchemy.engine.Engine [cached since 99.65s ago] (1,)
tag1 = sess.query(Hashtag).where(Hashtag.name == '해시태그2').one()
2023-09-07 11:56:24,162 INFO sqlalchemy.engine.Engine UPDATE "HASHTAG" SET "CNT"=? WHERE "HASHTAG"."PK" = ?
2023-09-07 11:56:24,165 INFO sqlalchemy.engine.Engine [cached since 132.2s ago] (2, 1)
2023-09-07 11:56:24,167 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT"
FROM "HASHTAG"
WHERE "HASHTAG"."NAME" = ?
2023-09-07 11:56:24,168 INFO sqlalchemy.engine.Engine [cached since 32.52s ago] ('해시태그2',)
for post in tag1.posts:
    print(post.post.content)
2023-09-07 11:57:02,282 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:57:02,288 INFO sqlalchemy.engine.Engine [cached since 170.3s ago] (1,)
내용2
2023-09-07 11:57:02,291 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT"
FROM "POST"
WHERE "POST"."PK" = ?
2023-09-07 11:57:02,292 INFO sqlalchemy.engine.Engine [cached since 170.3s ago] (2,)
내용3
tag1
1, 해시태그2, 2